In [3]:
import pandas as pd

df=pd.read_csv("../csv_files/daily_energy_values.csv")
df


,date,nea,nea_subsidiary,ipp,import,total_energy_available,energy_export,inps_demand,energy_interruption,generation_deficit,energy_requirement,net_exchange_with_india
0,2022/09/15,9482,12503,18521,11,40517.0,9316,31201,0,0,31201,0
1,2022/09/26,9302,12852,20023,264,42441.0,8913,33528,0,0,33528,0
2,2022/09/11,10073,12887,19842,788,43589.0,8736,34853,0,0,34853,0
3,2022/09/18,6837,12141,19040,147,38164.0,8295,29870,0,0,29870,0
4,2022/09/17,6682,3948,17978,79,28687.0,5323,23364,260,0,23624,0
...,...,...,...,...,...,...,...,...,...,...,...,...
846,2025/01/7,7376,5296,15231,7438,35340.0,0,35340,1550,0,36890,7438
847,2025/01/8,7160,5204,15439,7421,35225.0,0,35225,1950,0,37175,7421
848,2025/01/9,7580,5176,15658,7723,36137.0,0,36137,1800,0,37937,7723
849,2025/01/10,7192,5101,15642,7554,35489.0,0,35489,2100,0,37589,7554
